# Retrieve Trending Venues in Virginia Zip Codes and Save to CSV
Need to do this distributed throughout the day on three different days to get a picture of what venues are trending over time.

### This code was run on the following dates and times to create a data set of trending venues by zip code for use in further analysis:

* MM/DD/YY - HH:MM
* MM/DD/YY - HH:MM
* MM/DD/YY - HH:MM
* MM/DD/YY - HH:MM
* MM/DD/YY - HH:MM
* MM/DD/YY - HH:MM
* MM/DD/YY - HH:MM
* MM/DD/YY - HH:MM
* MM/DD/YY - HH:MM

In [1]:
# import needed libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


#!pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Below will be my FourSquare credentials. NEED TO REMOVE BEFORE SHARING.

In [2]:
CLIENT_ID = 'FHLIENCAGLQPDQRTXZFUBNVHPMW4TRXLFQN0OLHZ3SJXQGQF' # your Foursquare ID
CLIENT_SECRET = 'VT2WIU3QJHHQUFY04FC4BKHZJIMUA1TNZE4TAKYYUX4QGAEJ' # your Foursquare Secret
ACCESS_TOKEN = 'FTXV3LVR025425XYTLL0MHYQVJ4YOSZMCLH11I00PQGMGW42' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

Use the VirginiaPopByZipCode CSV to get all the Virginia Zip codes that will be used to retrieve trending venues.

In [3]:
df_ZipPop = pd.read_csv(r'Data/VirginiaPopByZipCode.csv')
print(df_ZipPop.shape)
df_ZipPop.head()

(892, 4)


,zip,city,county,pop
0,22193,Woodbridge,Prince William,82573
1,23464,Virginia Beach,Virginia Beach City,76495
2,22191,Woodbridge,Prince William,70641
3,23322,Chesapeake,Chesapeake City,65603
4,20147,Ashburn,Loudoun,64197


In [20]:
# need to change data type for later merging
df_ZipPop['zip'] = df_ZipPop['zip'].astype(str)
print(df_ZipPop.dtypes)

zip       object
city      object
county    object
pop        int64
dtype: object


### For each zip code:

1. get the associated Lat Long
2. get a list of trending venues
3. add the values **with categories** to a DataFrame
4. save the resulting DataFrame to a datetime specific csv for later use.

### Get Lat Lon for each Zip Code

Use the Open Data Standard API to retrieve lat lon values for every Virginia Zip Code.

In [5]:
#https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=VA&rows=1275&facet=state&facet=timezone&facet=dst&refine.state=VA

zip_url = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q=VA&rows=1275&facet=state&facet=timezone&facet=dst&refine.state=VA'
            
# make the GET request
zip_results = requests.get(zip_url).json()['records']
#print(zip_results)
zip_latlon_list=[]
zip_latlon_list.append([(z['fields']['zip'], z['fields']['latitude'], z['fields']['longitude']) for z in zip_results])

df_ZipLatLon = pd.DataFrame([item for zip_latlon_list in zip_latlon_list for item in zip_latlon_list])
df_ZipLatLon.columns = ['zip', 'Latitude', 'Longitude']

In [19]:
print(df_ZipLatLon.shape)
df_ZipLatLon['zip'] = df_ZipLatLon['zip'].astype(str)
print(df_ZipLatLon.dtypes)
df_ZipLatLon.head()

(1275, 3)
zip           object
Latitude     float64
Longitude    float64
dtype: object


,zip,Latitude,Longitude
0,23887,36.563755,-77.82652
1,24656,37.198005,-82.12193
2,24149,37.011934,-80.41897
3,23125,37.342721,-76.27989
4,22652,38.823987,-78.45310


Merge the lat lon DataFrame with the Population Data Frame


In [21]:
df_ZipUnion = df_ZipLatLon.merge(df_ZipPop, on='zip', how='left')

In [25]:
df_ZipUnion.dropna(inplace=True)
print(df_ZipUnion.shape)
df_ZipUnion.head()

(891, 6)


,zip,Latitude,Longitude,city,county,pop
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0
3,23125,37.342721,-76.27989,New Point,Mathews,255.0
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0


Create new areName column for friendly reference:

In [27]:
df_ZipUnion['areaName'] = df_ZipUnion['city'] + ', ' + df_ZipUnion['county'] + " (" + df_ZipUnion['zip'] + ')'
df_ZipUnion.head()

,zip,Latitude,Longitude,city,county,pop,areaName
0,23887,36.563755,-77.82652,Valentines,Brunswick,387.0,"Valentines, Brunswick (23887)"
1,24656,37.198005,-82.12193,Vansant,Buchanan,2812.0,"Vansant, Buchanan (24656)"
2,24149,37.011934,-80.41897,Riner,Montgomery,3530.0,"Riner, Montgomery (24149)"
3,23125,37.342721,-76.27989,New Point,Mathews,255.0,"New Point, Mathews (23125)"
4,22652,38.823987,-78.45310,Fort Valley,Shenandoah,1339.0,"Fort Valley, Shenandoah (22652)"


### Get Trending Venues

In [73]:
names = df_ZipUnion['areaName']
latitudes = df_ZipUnion['Latitude']
longitudes = df_ZipUnion['Longitude']
radius = 2000

In [58]:
venues_list = []

In [84]:
    
for name, lat, lng in zip(names, latitudes, longitudes):
    #print(name)
        
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/trending?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
        
    try :
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])
    except:
        venues_list.append([('NaN','NaN','NaN','NaN','NaN','NaN','NaN')])    

In [86]:
venues_list

[[('NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN')],
 [('NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN')],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [

In [88]:
trending_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
trending_venues.columns = ['Location', 
                'Latitude', 
                'Longitude', 
                'Venue', 
                'Venue Latitude', 
                'Venue Longitude', 
                'Venue Category']

### Get all the trending venues for the lat lons of the zip codes:

In [89]:
print(trending_venues.shape)
trending_venues.head()

(2, 7)


,Location,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Explore venues that are close to the zip code

In [79]:
explore_venues_list = []

for name, lat, lng in zip(names, latitudes, longitudes):
     
    # create the API request URL
    explore_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

    try:
        # make the GET request
        explore_results = requests.get(explore_url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        explore_venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in explore_results])
    except:
        explore_venues_list.append([('NaN','NaN','NaN','NaN','NaN','NaN','NaN')])

  

In [81]:
nearby_venues = pd.DataFrame([item for explore_venues_list in explore_venues_list for item in explore_venues_list])
nearby_venues.columns = ['Location', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [85]:
print(nearby_venues.shape)
nearby_venues.head()

(8640, 7)


,Location,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Riner, Montgomery (24149)",37.011934,-80.41897,bread basket,37.018349,-80.428886,Bakery
1,"Riner, Montgomery (24149)",37.011934,-80.41897,B & B Quality Fencing Inc,37.023586,-80.426829,Moving Target
2,"New Point, Mathews (23125)",37.342721,-76.27989,New Point RV Resort,37.344380,-76.276341,RV Park
3,"New Point, Mathews (23125)",37.342721,-76.27989,Warung Kopi Fauzan Lampineung,37.353405,-76.285563,Café
4,"New Point, Mathews (23125)",37.342721,-76.27989,Tower Coffee,37.353405,-76.285563,Coffee Shop


Save the Explore Venues to a csv

In [83]:
nearby_venues.to_csv('ExploreVenues_2020_12_28_0338PM.csv')